# syntax tools




## locating libraries



### get requires from source?

get requires?

throws an error on dynamic includes.


#### the code

In [ ]:
var importer = require('../Core');
var {selectAst} = importer.import('select code tree')

// TODO: use this in files and gulp script 
//   require("module").builtinModules
function getRequires(code) {
    return [].concat.apply([], selectAst([
        `//CallExpression[./Identifier[@name="require"]]`,
        (ctx) => {
            var req = selectAst([`.//Literal/@value`], ctx)
            if(req.length === 0) throw new Error(
                `dynamic require: ${JSON.stringify(htmlToTree(ctx))}`)
            return req;
        }
    ], code))
}


module.exports = {
    getRequires
}


#### test require detection



In [ ]:
var importer = require('../Core');
var {getRequires} = importer.import('get requires from source')

var code = `
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
`

function testGetRequires() {
    return getRequires(code)
}

module.exports = testGetRequires

if(typeof $$ != 'undefined') {
    testGetRequires()
    
    /*
    expected output
    ../Core
    glob
    path
    */
}


#### builtin and local modules?



In [ ]:
var module = require('module')
var importer = require('../Core')
var {getRequires} = importer.import('get requires')

function relativeImports(code, pathToCode) {
    var result = {
        local: [],
        builtin: [],
        packages: [],
        missing: []
    }
    var requires = getRequires(code)
    requires.forEach(imp => {
        var local = imp.substr(0, 1) === '.'
        try {
            if(local) {
                imp = path.relative(__dirname, pathToCode, imp)
                require.resolve(imp)
                result.local.push(imp)
            } else {
                if(!module.builtinModules.includes(imp)) {
                    throw new Error('Cannot find module')
                }
                result.builtin.push(imp)
            }
        } catch (e) {
            if(!e.message.includes('Cannot find module')) {
                throw e
            } else {
                if(local) {
                    result.missing.push(imp)
                } else {
                    result.packages.push(imp)
                }
            }
        }
    })
    return result
}

module.exports = {
    relativeImports
}


#### show all package dependencies in core

core dependencies?

Needs to be zero, and only builtins used.



In [ ]:
var path = require('path')
var importer = require('../Core')
var {relativeImports} = importer.import('builtin and local modules')
var {listInProject} = importer.import('list project files')

function coreDependencies() {
    var packages = []
    var builtin = []
    var coreNotebooks = listInProject(__dirname, '{,*,*/,*/*/*,*/*/*/*}*.ipynb')
        .filter(n => !n.includes('cache.ipynb'))
    coreNotebooks.forEach(n => {
        var cells = importer.interpret(path.basename(n))
        cells.forEach(cell => {
            try {
                var imports = relativeImports(cell.code, cell.filename)
                imports.packages.forEach(p => packages.push(p))
                imports.builtin.forEach(p => builtin.push(p))
            } catch (e) {
                console.log(`problem with ${cell.id} ${e.message}`)
            }
            
        })
    })
    console.log(packages)
    console.log(builtin)
    return packages
}

module.exports = {
    coreDependencies
}


#### test core dependencies?



In [ ]:
var importer = require('../Core')
var {coreDependencies} = importer.import('core dependencies')

function testCoreDependencies() {
    return coreDependencies()
}

module.exports = testCoreDependencies

if(typeof $$ !== 'undefined') {
    testCoreDependencies()
}


### get exports from source?

get module exports from source?


#### the code



In [ ]:
var importer = require('../Core');
var {selectAst} = importer.import('select code tree')

// TODO: does this work on other node_modules?
function getExports(code) {
    var exports = [].concat.apply([], selectAst([
        `//AssignmentExpression[.//Identifier[@name="exports"]]`,
        [`.//Identifier/@name`]
    ], code))
    var functions = selectAst([
        `//FunctionDeclaration/Identifier[@parent-attr="id"]/@name`
    ], code)
    return exports.concat(functions)
        .filter(e => e !== 'exports' && e !== 'require' && e !== 'module')
        .filter((e, i, arr) => arr.indexOf(e) === i)
}

module.exports = getExports;


#### test export detection



In [ ]:
var importer = require('../Core');
var getExports = importer.import('get exports from source')

var code = `
exports.import = importNotebook
exports = testExports
exports = require('../index.js')
`

function testGetExports() {
    return getExports(code)
}

if(typeof $$ != 'undefined') {
    testGetExports()
    
    /*
    expected output
    importNotebook
    import
    testExports
    */
}


### get parameter names?

function parameters?


#### the code



In [ ]:
var importer = require('../Core');
var {getExports, selectAst} = importer.import([
    'select code tree', 'get exports from source'])

var EXPORTS = `//AssignmentExpression[.//Identifier[@name=\"exports\"]]//Identifier/@name`
var PARAMETERS = `(//FunctionDeclaration|//ArrowFunctionExpression)[
    ./Identifier[@parent-attr=\"id\" and @name=(${EXPORTS})]
]/*[@parent-attr=\"params\"]/@name`

function getParameters(code) {
    var fnName = getExports(code)[0];
    if(!fnName) {
        return [];
    }
    var params = selectAst([PARAMETERS], code)
    return [fnName, ...params];
}

module.exports = getParameters;


#### test parameter names?



In [ ]:
var importer = require('../Core');
var getParameters = importer.import('get parameter names')

var code = `
function getParameters(code) {
}
`

function testGetParameters() {
    return getParameters(code)
}

if(typeof $$ != 'undefined') {
    testGetParameters()
    
    /*
    expected output
    getParameters
    code
    
    */
}


### get imports from source?



#### the code



In [ ]:
var importer = require('../Core');
var {selectAst} = importer.import('select code tree')
var CORE_DECLARE = `//*[
    contains(@type, "Declar")
    and ./*/*[@type="Literal" and contains(@value, "Core")]
]`;
var IMPORTER = `${CORE_DECLARE}/*[@type="Identifier"]/@name`;
var IMPORTER_CALLS = `//*[contains(@type, "Call") and ./*[
    contains(@type, "Member") and contains(./Identifier/@name, "import")
] and ./Literal]`;

function getImports(code) {
    var importer = selectAst([IMPORTER], code)[0]
    return [].concat.apply([], selectAst([
        IMPORTER_CALLS.replace('importer', importer),
        [`./Identifier/@name|./*/Identifier/@name|.//Literal/@value`]
    ], code))
        .filter((i, j, arr) => arr.indexOf(i) === j)
}

module.exports = getImports;



#### test get imports?



In [ ]:
var importer = require('../Core');
var getImports = importer.import('get imports from source')
var {selectAst} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')

var code = `
var importer = require('../Core');
var getParameters = importer.import('get parameter names')
`

function testGetImports() {
    console.log(getImports(importer.interpret('rpc.ipynb[1]').code))
    console.log(htmlToTree(selectAst('//CallExpression', code)[1]))
    return getImports(code)
}

if(typeof $$ != 'undefined') {
    testGetImports()
    
    /*
    expected output
    get parameter names
    import
    
    */
}


### TODO: retrieving framework documentation

generate a synax tree from documentation



#### the code

#### test language translator

TODO: Creating an interface like this becomes a simple http request to download and create a language compatible pattern, do in C# and in Node




In [ ]:
/*
var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request, inspect_request, complete_request,
    history_request, is_complete_request, shutdown_request,
    // requests maybe not required right now?
    kernel_info_request, interrupt_request, input_request,
    connect_request, comm_info_request, kernel_info_request,

    // implement all replys
    execute_reply, inspect_reply, complete_reply,
    history_reply, kernel_info_reply, is_complete_reply,
    connect_reply, comm_info_reply, kernel_info_reply,
    shutdown_reply, interrupt_reply, input_reply,

    // a few extra protocol methods
    display_data, update_display_data, execute_input,
    execute_result, error, status, clear_output,
    comm_msg, comm_close
}

module.exports = wireKernelInterface;
*/


### TODO: creating libraries from code blocks

Code analysis to combine blocks into modules


### TODO: manipulating code libraries

TODO: show git repo organization options

Warn of changes made outside of the module or feature branch



## code trees

Functions for working with code trees. selectTree is nice, but jsel and CSS-style selectors don't give you access to parents. True XPath can navigate both directions of an XML/HTML tree. Later in the section selectAst converts the code to an html tree for XPath and CSS-style evaluating.

### tree to html

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons. XML, HTML, and CSS parsers are common in every language, which means the XPath and CSS query selectors also might be available.

Therefore we have an AST that acts exactly the same in every language, no need to change class names, or make switch blocks for different token types.


#### the code

tree to html?


In [ ]:
var importer = require('../Core');
var {selectDom} = importer.import('select tree')
var {JSDOM} = require('jsdom');

//TODO: remove \u200b characters
var specialChars = (str) => {
    var special = {
        '&':'&amp;',
        '<':'&lt;',
        '>':'&gt;',
        '"':'&quot;',
        "'":'&apos;'
    }
    Object.keys(special).forEach(s => {
        str = str.replace(new RegExp(s, 'ig'), special[s])
    })
    return str
}

function treeToStr(statement, parent) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += treeToStr(statement[i], parent)
            continue;
        }
        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() treeToStr(() => el.type)
        if(statement[i] && typeof statement[i].type !== 'undefined'
           // or if the property is an Array,
           //   print them out as child elements
           || (Array.isArray(statement[i]) && statement[i].length
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `
${treeToStr(statement[i], i)}`
            // output as child element and <-attr> elements
            //   for posterity, only child elements are return on the select,
            //   but attr elements can still be matched
        } else {
            // if the property is not an object, i.e. native types
            if(typeof statement[i] !== 'string') {
                attrs +=  `
${i}="${specialChars(JSON.stringify(statement[i]))}" ${i}-type="${jsType}"`
            } else {
                attrs +=  `
${i}="${specialChars(statement[i])}" ${i}-type="${jsType}"`
            }
        }
    }
    
    return isList ? result : `
<${type}${attrs}${parent ? `
 parent-attr="${parent}"`: ``}>${result}</${type}>`
}

function treeToHtml(tree) {
    var body = treeToStr(tree);
    return selectDom('//BODY', '<BODY>' + body + '</BODY>');
}

module.exports = {
    treeToHtml
}


#### justification

Just so it can be parsed with XPath instead of crappy wanna-be query languahes. Get them all working the same!  Also, html/xml parsers are common in many languages. So this library can be parsed and translated to other languages, using itself!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonpath-plus
- jsonquery
- jspath
- json:select
- jsel
- minimatch

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



#### test converting element to markdown using an ast



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### html to tree

Convert HTML back from the function above, html to tree.

#### the code

html to tree?

In [ ]:
var {JSDOM} = require('jsdom');

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateChildNodes(body) {
    var commentBuffer = []
    // TODO: exclude children parent properties
    //   left-over children are assigned to children: []
    return Array.from(body.childNodes)
        .reduce((obj, n) => {
            if(n.nodeName === '#text') {
                commentBuffer.push[n];
                return obj;
            }
            var parent = n.getAttribute('parent-attr');
            var newNode = htmlToTree(n);
            if(parent) {
                var parentType = body.getAttribute(parent + '-type')
                if(parentType === 'Array') {
                    if(typeof obj[parent] === 'undefined')
                        obj[parent] = [];
                    obj[parent].push(newNode);
                } else {
                    obj[parent] = newNode;
                }
            } else {
                // TODO: if no children left and no other -attr properties,
                //   remove the child property from the output
                if(typeof obj.children === 'undefined')
                    obj.children = [];
                obj.children.push(newNode);
            }
            if(typeof newNode.comments !== 'undefined') {
                newNode.comments.push.apply(
                    newNode.comments, commentBuffer)
                commentBuffer = [];
            }
            return obj;
        }, {})
}

// expects a string or a tree from JSDOM
function htmlToTree(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        return accumulateChildNodes(dom.window.document.body).children;
    }
    if(Array.isArray(body)) {
        return body.map(t => htmlToTree(t))
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    // convert attributed object containers back to properties
    // TODO: might have a property named -type and it will be lost
    var attrs = body.getAttributeNames()
        .filter(a => a !== 'parent-attr' && a.substr(-5) != '-type')
        .reduce((obj, p) => {
            var attr = body.getAttribute(p);
            try {
                if(body.getAttribute(`${p}-type`) !== 'string') {
                    obj[p] = JSON.parse(attr);
                } else {
                    obj[p] = attr;
                }
            }
            catch (e) { obj[p] = attr; }
            return obj;
        }, {})
    
    var children = accumulateChildNodes(body);
    
    return Object.assign({
        type: body.nodeName
    }, attrs, children);
}

module.exports = {
    htmlToTree
};



#### test tree to html



In [ ]:
var importer = require('../Core');
var {treeToHtml, htmlToTree} = importer.import([
    'tree to html', 'html to tree']);

var assert = require('assert');
var esprima = require('esprima'); 

function setupASTArray(code) {
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    return ctx;
}

function testASTArray(ctx) {
    var page = treeToHtml(ctx);
    var translated_back = htmlToTree(page);
    console.log(translated_back[0]);
    assert(JSON.stringify(ctx).length
           === JSON.stringify(translated_back[0]).length,
           'different code trees');    
    console.log('two JSON trees are same-same')
    return page;
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = setupASTArray(code);
    var html = testASTArray(ctx);
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


#### test what happens on regular html converted to a tree




In [ ]:
var importer = require('../Core');
var {htmlToTree} = importer.import('html to tree');

function testHtmlTree(page) {
    var translated_back = htmlToTree(page);
    return translated_back;
}

if(typeof $$ !== 'undefined') {
    // copied from Chrome Google search homepage
    var code = `<div id="fakebox-container">
      <div id="fakebox">
        <div id="fakebox-search-icon"></div>
        <div id="fakebox-text">Search Google or type a URL</div>
        <input id="fakebox-input" autocomplete="off" tabindex="-1" type="url" aria-hidden="true">
        <div id="fakebox-cursor"></div>
        <button id="fakebox-microphone" title="Search by voice"></button>
      </div>
    </div>`
    var tree = testHtmlTree(code);
    console.log(tree);
}


### select code tree?

Select code from tree?

Use CSS selectors to find code.



#### the code


In [ ]:
var esprima = require('esprima')
var escodegen = require('escodegen')
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])

var FUNCTION_BODY = `//FunctionDeclaration/BlockStatement`

function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectAst(descriptor, code) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        code = esprima.parse(code, {range: true, tokens: true,
                                    comment: true, whitespace: true})
    }
    // TODO: move this conversion to selectDom for all objects?
    //   No more selectTree because jsel and friends are inconsistent.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

function makeExpr(code) {
    if(typeof code === 'function') {
        return selectAst([FUNCTION_BODY], code.toString())[0]
    }
    return selectAst([`//ExpressionStatement/*`], `(${code})`)[0]
}

module.exports = {
    selectAst,
    makeExpr
}


#### test selecting xpath with parent data

Test selecting trees with the html translator.

Get the same result as using the json evaluator.


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {selectAst, makeXpaths, htmlToTree} = importer.import([
    'select tree', 'make xpaths', 'html to tree'])

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testCodeToDom(code) {
    // make a path with the interpret symbol
    var node1 = selectAst(`//*[@name="interpret"]`, code)
    console.log(node1)
    var parent = selectAst(`parent::*`, node1);
    assert(parent.nodeName !== '', 'parent selector works');
    console.log(selectAst(`*`, node1).children.length);
    console.log(htmlToTree(parent));
}

if(typeof $$ !== 'undefined') {
    testCodeToDom(code);
}


#### test with acorn instead

select acorn tree?


In [ ]:
var acorn = require("acorn")
var importer = require('../Core')
var {selectAst, htmlToTree} = importer.import([
    'select code tree', 'html to tree'])

function selectAcorn(descriptor, code) {
    var comments = [], tokens = [];
    if(typeof code === 'string') {
        code = acorn.parse(code, {
            ecmaVersion: 6,
            // collect ranges for each node
            locations: true,
            ranges: true,
            // collect comments in Esprima's format
            onComment: comments,
            // collect token ranges
            onToken: tokens,
        })
    }
    return selectAst(descriptor, code);
}

module.exports = {
    selectAcorn
}

if(typeof $$ !== 'undefined') {
    var code = `
    var importer = require('../Core');
    function name(params) {
        return importer.interpret('this is a describe request');
    }
    console.log()
    `
    var call = selectAcorn(`//*[@name="interpret"]`, code);
    console.log(htmlToTree(call))
}


### get xpath to node

XPath minimizing service from recording prototype app.
 

#### make xpaths?

For generating links to nodes from the node, uses parentElement.



In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {selectDom} = importer.import('select tree');

function makeCombinations(values) {
    // get every combination of matching classifiers
    //   pass it up to be used for minimization
    //   on a larger context
    var combinations = [];
    // start at 1 to always include * or Element tag name
    let i, j;
    for (i = 1; i <= values.length; i++) {
        for (j = 1; j <= values.length; j++) {
            combinations.push([
                values[0],
                ...values.slice(i, j)
            ].join(''))
        }
    }
    return combinations
        .filter((c, i, arr) => arr.indexOf(c) === i)
        .sort((a, b) => a.length - b.length)
}

// algorithm magic!
// create an very specific XPath following these rules
function makeXpaths(node) {
    var classifiers = [{
        tag: `local-name(.)`,
        ids: [`@*[name()="type" or name()="id" or name()="name"]`],
        classNames: `@class`,
        index: `count(preceding-sibling::*)`,
        parent: `local-name(parent::*)`,
        //parent: `parent::*/@*[name()="type" or name()="id" or name()="name"]`,
        nthType: `count(preceding-sibling::*[local-name(*)=local-name(.)])`
    }, ({tag, ids, classNames, index, parent, nthType}) => ({
        tag: tag && tag !== 'object' ? tag : `*`,
        id: ids.length
            ? `[${ids
            .map(id => 
                 `(@id="${id}" or @name="${id}" or @type="${id}")`)
            .join(` and `)}]`
            : ``,
        classNames: classNames.length > 0
            ? `[contains(@class, "${classNames}")]`
            : ``,
        index: typeof index !== 'undefined' && parent
            ? `/parent::${parent}/*[${index+1}]`
            : ``,
        parent: parent && typeof nthType !== 'undefined'
            ? `/parent::${parent}/${
                tag && tag !== 'object' ? tag : `*`}[${nthType+1}]`
            : ``
    })]
    
    var classifiers = selectDom(classifiers, node)[0];
    var {tag, id, classNames, index, parent} = classifiers;
    // to ensure the order from most specific to least specific
    var combinations = [].concat.apply([], makeCombinations( 
                                       [tag, id, classNames, index, parent]))
    assert(selectDom(combinations[0], node),
           'oops, the xpath expression maker messed up');
    return combinations;
}

module.exports = {
    makeXpaths
};


#### test making an xpath from any arbitraty node

generate css selectors from code?

problem, jsel doesn't support parent()


In [ ]:
var esprima = require('esprima'); 
var assert = require('assert');
var importer = require('../Core');
var {selectAst, makeXpaths, htmlToTree} = importer.import([
    'select code tree', 'make xpaths', 'html to tree'
])

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testMakeXpaths(code) {
    // make a path with the interpret symbol
    var node1 = selectAst(`//*[@name="name"]`, code)
    var parent = selectAst(`//*[@type="FunctionDeclaration"]`, node1);
    var output = makeXpaths(parent)
    var node2 = selectAst(`//${output[0]}`, code)
    console.log(node2)
    assert(node1.parentNode.getAttribute('name') === node2.getAttribute('name'));
    return node2
    
}
// TODO: compare with acorn
if(typeof $$ !== 'undefined') {
    var nodes = testMakeXpaths(code);
    console.log(htmlToTree(nodes));
}


#### minimize xpath?

Should get something like ```Identifier[@name="importer"]``` by the end of this.



In [ ]:
var importer = require('../Core');
var {selectDom} = importer.import('select tree');

function minXpath(combinations, ctx) {
    // flatten the XPath stack using '/'
    // sort by smallest and return shortest path matching 1 DOM element
    const minimal = combinations
            .filter(c => selectDom([`.//${c}`], ctx).length === 1)
            .sort((a, b) => a.length - b.length);
    return minimal[ 0 ];

    // TODO: prioritize by button/input, ids, classes, attributes? (extra credit), index
}

module.exports = {
    minXpath
};


#### TODO: convert simple xpath to css selector

In [ ]:

function convertXPathToCss(path) {
    // if matched xpath is simple enough, convert it to CSS
    // DIV[contains(@class, "product-tile")]/parent::*/DIV[2]//MD-CARD[contains(@class, "mat-card")]/parent::*/MD-CARD[1]
    return path
            .replace(/\/([a-z-]+)\[@id="(.*?)"]/ig, '/$1#$2')
            .replace(/\/([^\/]+)\[contains\(@class, "(.*?)"\)]/ig, '/$1.$2')
            .replace(/\/parent::[a-z-]+\/[a-z-]+\[([0-9]+)]/ig, ':nth-child($1)')
            .replace(/^\/\//ig, '')
            .replace(/\/\//ig, ' ')
            .replace(/\//ig, ' > ');
}




### improved syntax manipulation

Replacing the functionality I noticed in babel somewhere. Basically walking a tree with visitors is ugly, so we replace it with a sequence of CSS style queries.



#### select expression?

select from code?

Convert the body of a function to a list of search strings.


In [ ]:
var importer = require('../Core');
var {
    selectAst, makeXpaths, minXpath,
    htmlToTree
} = importer.import([
    'select code tree', 'make xpaths', 'minimize xpath',
    'html to tree']);

function exprToXpath(code) {
    // create an XPath search out of the function body
    //   using the parameters as matching predicates
    code = selectAst([
        `(*/FunctionDeclaration|*/*/ArrowFunctionExpression)`
    ], code.toString())[0]
    
    var expression = selectAst([
        `Identifier[@parent-attr="params"]/@name`,
        // select all elements from func that match param name
        //   TODO: default functionality can be overridden by specifying
        //   a function as the default parameter value
        (name) => selectAst([`BlockStatement//*[@name="${name}"]`], code)[0]
    ], code)[0]; // TODO: make multiple expressions?
    if(!expression) {
        throw new Error(`can't match expression ${JSON.stringify(htmlToTree(code))}`)
    }
    var xpath = makeXpaths(expression);
    // add minimizing expressions, unique to the original context
    var min = minXpath(xpath, selectAst(`BlockStatement`, code));
    delete code;
    return min;
}

module.exports = {
    exprToXpath
}


#### test selecting expressions


In [ ]:
var importer = require('../Core');
var {exprToXpath} = importer.import('select expression');

if(typeof $$ !== 'undefined') {
    var xpath = exprToXpath((importer) => {
            var importer = require('../Core');
        });
    
    console.log(xpath)
//    replaceCore(code)
}



#### test expression on notebook code?

TODO: improve speed



In [ ]:
var importer = require('../Core');
var {exprToXpath, selectAst} = importer.import([
    'select expression', 'select code tree'
]);
var {cellCache} = importer.import('cell cache')

function matchCell(xpath, cell) {
    try {
        var match = selectAst([`//${xpath}`], cell.code);
        if(match.length > 0) {
            console.log(`match ${cell.id}`)
            return cell;
        }
        return false;
    } catch (e) {
        console.log(e.message);
        return false;
    }
}

function findImport(importer) {
    var importer = require('../Core');
}

function testExpressions() {
    var xpath = exprToXpath(findImport);
    console.log(`matching ${xpath}`);
    var allCellIds = cellCache.map(c => c.id)
    // get only first occurrence
    var allCells = cellCache
        .filter((c, i) => allCellIds.indexOf(c.id) == i)
        .filter(c => c.code.length < 10000
                && c.code.trim().length > 10)
    //    .slice(0, 10)
    return Promise
        .all(allCells.map(cell => new Promise(resolve => {
            return setTimeout(() => resolve(matchCell(xpath, cell)), 100);
        })))
        .then(matches => matches.filter(cell => cell)
              .map(cell => cell.id))
}

module.exports = testExpressions;

if(typeof $$ !== 'undefined') {
    testExpressions()
        .then(matches => $$.sendResult(matches))
}


#### TODO: test with odd code like convert spreadsheet[4]



## TODO: generate unit tests

Generate unit test from logic branching?

TODO: use language server for this

Attach to the same branches,

https://github.com/gotwarlost/istanbul/blob/master/lib/instrumenter.js



### unit test each token type



#### statements



In [ ]:
var STATEMENTS = `//*[contains(@type, "Declaration")]
|//*[contains(@type, "Statement")]`



#### calls



In [ ]:
var CALLS = `//CallExpression`



#### logical branches



In [ ]:

var BRANCHES = `//IfStatement|SwitchStatement`



### generate unit test?



#### the code



#### update notebook test cell

#### TODO: generate a test from expected output


## TODO: detecting framework features

Show router tree, module import tree, call tree, feature tree


## TODO: format detection by example

Lint all code

Reformat all code like when using webstorm
